# SAE-15: TP-1

In [3]:
from lxml import etree
from io import StringIO, BytesIO
import requests
import time
import os


In [ ]:
def checkDirIsPresent(directory: str, if_not_create: bool = False):
  if os.path.exists(os.path.abspath("./")+directory) == False:
    if if_not_create:
      if os.name == "nt":
        windir = directory.replace('/', '\\')
      os.system(f"mkdir -p {windir}")
      if checkDirIsPresent(directory):
       print(f"checkDirIsPresent: The directory '{directory}' has been successfully created")

  return True


def requestThenPrint(request: str):
  response = requests.get(request)
  print(response.text)


def requestThenReturn(request: str):
  response = requests.get(request)
  return response.text

def requestThenReturnAsFileObject(request: str):
  response = requests.get(request)
  response_text = response.text
  _file = BytesIO(bytes(response_text, encoding="utf-8"))
  return _file

def requestThenWrite(request: str, _dir: str = None):
  if _dir == None:
    _dir = "./in/stats"

  response = requests.get(request)
  filename = response.url.strip().split("/")[-1]

  if checkDirIsPresent(_dir, if_not_create=True):

    with open(f"{_dir}/{filename}", "xt", encoding="utf-8") as fout:
      fout.write(response.text)


def parseXMLData(parking: str):
  state = ""
  opened = False  # valeur "globale" dans le namespace de la fonction
  free_places = 0  # valeur "globale" dans le namespace de la fonction
  # On supprime l'extension du fichier afin d'avoir juste le nom - Utilisation future probablement
  # Création d'un fichier temporaire
  with open("temp-file.log", "wt", encoding="utf-8") as f_temp:
    f_temp.write(parking)
  tree = etree.parse("temp-file.log")
  # Condition tertiaire afin d'avoir une valeur boulléenne
  date = tree.xpath("DateTime")[0].text
  opened = True if tree.xpath("Status")[0].text.lower() == "open" else False
  free_places = int(tree.xpath("Free")[0].text)
  places = int(tree.xpath("Total")[0].text)
  name = tree.xpath("Name")[0].text

  state = f"{name},{date},{opened},{free_places},{places}"

  # On nettoie notre fichier temporaire - Garbage Collector
  os.remove("temp-file.log")

  return state


def isCSVHeaderPresentIfNotWrite(filename : str, csv_header: str):
  is_required_to_write = False
  entire_file = csv_header+"\n"

  if os.path.exists(filename):
    with open(filename, "rt", encoding="utf-8") as fin:
      content = fin.readlines()
      if len(content) >= 1:
        print(content, csv_header, True if content[0].replace("\n", "") == csv_header else False)
        if content[0].replace("\n", "") == csv_header:
          is_required_to_write = False

        else:
          is_required_to_write = True
          # On ajoute donc l'en-tête CSV au fichier que l'on va réecrire
          content.insert(0, csv_header+"\n")
          entire_file = content

  if is_required_to_write:
    with open(filename, "wt", encoding="utf-8") as fout:
      fout.write("".join(entire_file))

def requestThenWriteDataHistory(request: str, _dir: str = None):
  if _dir == None:
    _dir = "./in/stats/csv"

  response = requests.get(request)
  filename = response.url.strip().split("/")[-1].replace(".xml", ".csv")
  filename = filename.upper()

  if checkDirIsPresent(_dir, if_not_create=True):

    isCSVHeaderPresentIfNotWrite(f"{_dir}/{filename}", "name,date,opened,free_places,places")

    with open(f"{_dir}/{filename}", "at", encoding="utf-8") as fout:
      fout.write(parseXMLData(response.text) + "\n")


start_timestamp = int(time.time())
jump = 10
time_to_run = 60
print("Timstamp as of start:", start_timestamp)

while int(time.time()) != start_timestamp+time_to_run:
  while int(time.time()) == start_timestamp+jump:
    requestThenWriteDataHistory("https://data.montpellier3m.fr/sites/default/files/ressources/FR_MTP_GARE.xml")
    jump += 10

### Question 1 - Exécuter le code python suivant et expliquer ce qu’il fait:

In [5]:
print("Question 1:")
requestThenPrint("https://data.montpellier3m.fr/sites/default/files/ressources/FR_MTP_COME.xml")

Question 1:
<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2022-08-01T15:32:58</DateTime>
	<Name>COME</Name>
	<Status>Open</Status>
	<Free>0174</Free>
	<Total>0774</Total>
</park>



### Question 2 - On peut remarquer que le lien pour un parking est une chaine de caractères constituée de trois parties : 

In [6]:
print("\n\nQuestion 2:")
parkings = ['FR_MTP_ANTI.xml','FR_MTP_COME.xml','FR_MTP_CORU.xml','FR_MTP_EURO.xml','FR_MTP_FOCH.xml','FR_MTP_GAMB.xml','FR_MTP_GARE.xml','FR_MTP_TRIA.xml','FR_MTP_ARCT.xml','FR_MTP_PITO.xml','FR_MTP_CIRC.xml','FR_MTP_SABI.xml','FR_MTP_GARC.xml','FR_MTP_SABL.xml','FR_MTP_MOSS.xml','FR_STJ_SJLC.xml','FR_MTP_MEDC.xml','FR_MTP_OCCI.xml','FR_CAS_VICA.xml','FR_MTP_GA109.xml','FR_MTP_GA250.xml','FR_CAS_CDGA.xml','FR_MTP_ARCE.xml','FR_MTP_POLY.xml']

for parking in parkings :
  requestThenPrint(f"https://data.montpellier3m.fr/sites/default/files/ressources/{parking}")



Question 2:
<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2023-01-11T18:44:56</DateTime>
	<Name>ANTI</Name>
	<Status>Open</Status>
	<Free>0149</Free>
	<Total>0239</Total>
</park>

<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2022-08-01T15:32:58</DateTime>
	<Name>COME</Name>
	<Status>Open</Status>
	<Free>0174</Free>
	<Total>0774</Total>
</park>

<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2023-01-11T18:44:56</DateTime>
	<Name>CORU</Name>
	<Status>Open</Status>
	<Free>0038</Free>
	<Total>0440</Total>
</park>

<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2023-01-11T18:44:57</DateTime>
	<Name>EURO</Name>
	<Status>Open</Status>
	<Free>0425</Free>
	<Total>0593</Total>
</park>

<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2023-01-11T18:44:59</DateTime>
	<Name>FOCH</Name>
	<Status>Open</Status>
	<Free>0128</Free>
	<Total>0570</Total>
</park>

<?xml version='1.0' encoding='UTF-8'?>
<park>
	<DateTime>2023-01-11T18:44:56</DateTime

## Partie 2: analyse d'un fichier XML

> Dans la partie précédente, nous avons récupéré les données utiles provenant des différents parkings 
> de la ville. Les données sont fournies sous la forme d’un fichier xml.  
> Afin de récupérer une donnée précise (par exemple le nombre de places libres dans un parking, on 
> peut, bien entendu, traiter la chaine de caractère obtenue afin de récupérer la donnée utile. 
> Cependant, il est possible d’utiliser une librairie pour effectuer cette tâche. On parle alors de « parser » 
> ou analyser un fichier .xml.  
> Dans ce but, nous allons utiliser la librairie « lxml ». 

### Question 3 - Etudier le programme suivant et décrire ce qu’il effectue

In [7]:
response=requests.get("https://data.montpellier3m.fr/sites/default/files/ressources/FR_MTP_COME.xml")

print(response.text)
f1=open("FR_MTP_COME.txt","w", encoding='utf8')
f1.write(response.text)
f1.close()
tree = etree.parse("FR_MTP_COME.txt")

for user in tree.xpath("Name"):
  print('Nom du parking :', user.text)
for user in tree.xpath("Total"):
  print('Nombre total de places :', user.text)
for user in tree.xpath("Free"):
  print('Nombre de places libres :', user.text)


<?xml version='1.0' encoding='UTF-8'?> 
<park>
	<DateTime>2022-08-01T15:32:58</DateTime>
	<Name>COME</Name>
	<Status>Open</Status>
	<Free>0174</Free>
	<Total>0774</Total>
</park>

Nom du parking : COME
Nombre total de places : 0774
Nombre de places libres : 0174


Avant l'exécution, nous avons importé les bibliothèques python:

- `requests`: Nous permet d'émettres des requêtes (`GET`, en l'occurence)
- `lxml`: Nous permet de faire du parsing d'XML

En premier lieu, on effectue une requête GET avec le module requests puis on print sur la sortie standard le contenu du fichier (présent dans l'attribut `text`).  
Ensuite, on ouvre/crée (si non existant, même si l'option "*x*" n'est pas utilisée) un fichier du nom de `FR_MTP_COME.txt` avec un encodage en `UTF-8` dans le répertoire courant, on y écrit le contenu du corps de la réponse à notre requête GET puis on le ferme.

En dernier lieu, on ouvre ce fichier en utilisant la méthode `parse()` de `etree`, on fait ensuite trois boucles `FOR`:

- 1ère: On récupère le nom du parking (dans la balise `Name`), on en print: `Nom du parking : <nom-du-parking>` où `<nom-du-parking>` est le nom que l'on a récupéré précedemment.
- 2ème: On récupère le nombre total de places (dans la balise `Total`), on en print: `Nombre total de places : <nombre-total>` où `<nombre-total>` est le nombre de place total du parking que l'on requête.
- 3ème: On récupère le nombre de place libres (dans la balise `Free`), on en print: `Nombre de places libres : <places-libres>` où `<places-libres>` est le nombre de places libres dans le parking au moment de la requête.

### Question 4 - Ecrire un programme qui récupère le nombre de places libres dans chaque parking et qui sauvegarde ces données dans un fichier texte (une donnée par ligne contenant uniquement le nombre de places libres.

Il faudra vérifier pour chaque parking, s’il est ouvert. 

Avant le code, on utilisera la classe StringIO afin de ne pas créer de plus amples fichiers mais avoir une interface "Fichier-type" afin de pouvoir utiliser le module `etree` de `lxml`

In [8]:

def getOpenedParkingFreePlacesThenWrite(parking_list: list):
  ### _dict:
  ### {
  ###   "NAME": {
  ###     "open": <bool>,
  ###     "free": <int>
  ###     }
  ### }
  _dict = {} ### Utilisée à des fins de statistiques futures
  opened = False ### valeur "globale" dans le namespace de la fonction
  free_places = 0 ### valeur "globale" dans le namespace de la fonction
  for parking in parking_list:
    ### On supprime l'extension du fichier afin d'avoir juste le nom - Utilisation future probablement
    parking_name = parking.replace(".xml", "")
    response = requestThenReturn(f"https://data.montpellier3m.fr/sites/default/files/ressources/{parking}")
    ### Création d'un fichier temporaire
    with open("temp-file.log", "wt", encoding="utf-8") as f_temp:
      f_temp.write(response)
    tree = etree.parse("temp-file.log")
    ### Condition tertiaire afin d'avoir une valeur boulléenne
    opened = True if tree.xpath("Status")[0].text.lower() == "open" else False
    if opened:
      free_places = int(tree.xpath("Free")[0].text)
      ### On vérifie que le fichier existe
      ###
      ### Si oui, on ajoute la nouvelle valeur
      ### Si non, on crée le fichier et on ajoute la nouvelle valeur
      if os.path.exists("mtp-parkings-free-places.txt"):
        with open("mtp-parkings-free-places.txt", "at", encoding="utf-8") as fout:
          fout.write(f"{free_places}\n")
      else:
        with open("mtp-parkings-free-places.txt", "at", encoding="utf-8") as fout:
          fout.write(f"{free_places}\n")

    ### Utilisation future -> sortie de stats sous forme de dictionnaires (JSON-like)
    _dict.update({
      parking_name: {
        "open": opened,
        "free": free_places
      }
    })

  # On nettoie notre fichier temporaire - Garbage Collector
  os.remove("temp-file.log")

  return _dict

getOpenedParkingFreePlacesThenWrite(parkings)


{'FR_MTP_ANTI': {'open': True, 'free': 149},
 'FR_MTP_COME': {'open': True, 'free': 174},
 'FR_MTP_CORU': {'open': True, 'free': 38},
 'FR_MTP_EURO': {'open': True, 'free': 425},
 'FR_MTP_FOCH': {'open': True, 'free': 128},
 'FR_MTP_GAMB': {'open': True, 'free': 384},
 'FR_MTP_GARE': {'open': True, 'free': 222},
 'FR_MTP_TRIA': {'open': True, 'free': 144},
 'FR_MTP_ARCT': {'open': True, 'free': 104},
 'FR_MTP_PITO': {'open': True, 'free': 380},
 'FR_MTP_CIRC': {'open': True, 'free': 585},
 'FR_MTP_SABI': {'open': True, 'free': 186},
 'FR_MTP_GARC': {'open': True, 'free': 263},
 'FR_MTP_SABL': {'open': True, 'free': 245},
 'FR_MTP_MOSS': {'open': True, 'free': 281},
 'FR_STJ_SJLC': {'open': True, 'free': 143},
 'FR_MTP_MEDC': {'open': True, 'free': 204},
 'FR_MTP_OCCI': {'open': True, 'free': 468},
 'FR_CAS_VICA': {'open': True, 'free': 57},
 'FR_MTP_GA109': {'open': True, 'free': 99},
 'FR_MTP_GA250': {'open': True, 'free': 216},
 'FR_CAS_CDGA': {'open': True, 'free': 235},
 'FR_MTP_AR

### Question 5 - modifier votre programme pour sauvegarder un fichier contenant dans chaque ligne : le nom du parking puis le nombre de places libres.

In [9]:

def getOpenedParkingFreePlacesThenWrite(parking_list: list):
  ### _dict:
  ### {
  ###   "NAME": {
  ###     "open": <bool>,
  ###     "free": <int>,
  ###     "places": <int>
  ###   }
  ### }
  _dict = {}  # Utilisée à des fins de statistiques futures
  opened = False  # valeur "globale" dans le namespace de la fonction
  free_places = 0  # valeur "globale" dans le namespace de la fonction
  places = 0  # valeur "globale" dans le namespace de la fonction
  what_to_write = ""  # valeur "globale" dans le namespace de la fonction
  out_filename = "mtp-parkings-free-places-named.txt"  # valeur "globale" dans le namespace de la fonction
  for parking in parking_list:
    ### On supprime l'extension du fichier afin d'avoir juste le nom - Utilisation future probablement
    parking_name = parking.replace(".xml", "")
    response = requestThenReturn(f"https://data.montpellier3m.fr/sites/default/files/ressources/{parking}")
    ### Création d'un fichier temporaire
    with open("temp-file.log", "wt", encoding="utf-8") as f_temp:
      f_temp.write(response)
    tree = etree.parse("temp-file.log")
    ### Condition tertiaire afin d'avoir une valeur boulléenne
    opened = True if tree.xpath("Status")[0].text.lower() == "open" else False
    if opened:
      free_places = int(tree.xpath("Free")[0].text)
      places = int(tree.xpath("Total")[0].text)

      ### On définit à l'avance ce que l'on va écrire
      what_to_write = f"{parking_name} - {free_places}\n"

      ### On vérifie que le fichier existe
      ###
      ### Si oui, on ajoute la nouvelle valeur
      ### Si non, on crée le fichier et on ajoute la nouvelle valeur
      if os.path.exists(out_filename):
        with open(out_filename, "at", encoding="utf-8") as fout:
          fout.write(what_to_write)
      else:
        with open(out_filename, "at", encoding="utf-8") as fout:
          fout.write(what_to_write)

    ### Utilisation future -> sortie de stats sous forme de dictionnaires (JSON-like)
    _dict.update({
        parking_name: {
            "open": opened,
            "free": free_places,
            "places": places
        }
    })

  # On nettoie notre fichier temporaire - Garbage Collector
  os.remove("temp-file.log")

  return _dict

getOpenedParkingFreePlacesThenWrite(parkings)


{'FR_MTP_ANTI': {'open': True, 'free': 149, 'places': 239},
 'FR_MTP_COME': {'open': True, 'free': 174, 'places': 774},
 'FR_MTP_CORU': {'open': True, 'free': 38, 'places': 440},
 'FR_MTP_EURO': {'open': True, 'free': 425, 'places': 593},
 'FR_MTP_FOCH': {'open': True, 'free': 128, 'places': 570},
 'FR_MTP_GAMB': {'open': True, 'free': 384, 'places': 434},
 'FR_MTP_GARE': {'open': True, 'free': 222, 'places': 656},
 'FR_MTP_TRIA': {'open': True, 'free': 144, 'places': 436},
 'FR_MTP_ARCT': {'open': True, 'free': 104, 'places': 450},
 'FR_MTP_PITO': {'open': True, 'free': 380, 'places': 587},
 'FR_MTP_CIRC': {'open': True, 'free': 585, 'places': 1200},
 'FR_MTP_SABI': {'open': True, 'free': 186, 'places': 283},
 'FR_MTP_GARC': {'open': True, 'free': 263, 'places': 395},
 'FR_MTP_SABL': {'open': True, 'free': 245, 'places': 372},
 'FR_MTP_MOSS': {'open': True, 'free': 281, 'places': 350},
 'FR_STJ_SJLC': {'open': True, 'free': 143, 'places': 290},
 'FR_MTP_MEDC': {'open': True, 'free': 2

### Question 6 - écrire un programme qui donne le pourcentage de places libres pour chaque parking ainsi que le pourcentage de places libres dans toute la ville.

In [10]:
def getOpenedParkingFreePlacesStatistics(parking_list: list):
  ### _dict:
  ### {
  ###   "NAME": {
  ###     "open": <bool>,
  ###     "free": <int>,
  ###     "places": <int>
  ###   }
  ### }
  _dict = {}  # Utilisée à des fins de statistiques futures
  opened = False  # valeur "globale" dans le namespace de la fonction
  free_places = 0  # valeur "globale" dans le namespace de la fonction
  places = 0  # valeur "globale" dans le namespace de la fonction
  for parking in parking_list:
    ### On supprime l'extension du fichier afin d'avoir juste le nom - Utilisation future probablement
    parking_name = parking.replace(".xml", "")
    response = requestThenReturn(f"https://data.montpellier3m.fr/sites/default/files/ressources/{parking}")
    ### Création d'un fichier temporaire
    with open("temp-file.log", "wt", encoding="utf-8") as f_temp:
      f_temp.write(response)
    tree = etree.parse("temp-file.log")
    ### Condition tertiaire afin d'avoir une valeur boulléenne
    opened = True if tree.xpath("Status")[0].text.lower() == "open" else False
    if opened:
      free_places = int(tree.xpath("Free")[0].text)
      places = int(tree.xpath("Total")[0].text)

    ### Utilisation future -> sortie de stats sous forme de dictionnaires (JSON-like)
    _dict.update({
        parking_name: {
            "open": opened,
            "free": free_places,
            "places": places
        }
    })

  return _dict

def getCityData(parking_data: dict):
  ### _dict:
  ### {
  ###   "NAME": {
  ###     "open": <bool>,
  ###     "free": <int>,
  ###     "places": <int>
  ###   }
  ### }
  all_places = 0
  all_free_places = 0
  for parking in parking_data:
    all_places += parking_data[parking]["places"]
    all_free_places += parking_data[parking]["free"]
  return {
    "all_places": all_places,
    "all_free_places": all_free_places
  }

def getPercentageOfAvailability(stats: dict):
  availability = stats["all_free_places"]
  total = stats["all_places"]
  return f"{availability/total:.2%}"

### Faire un pourcentage
### libre*100/total
print(f"{846/1911.:.2%}")
montpellier_stats = getCityData(getOpenedParkingFreePlacesStatistics(parkings))
print(getPercentageOfAvailability(montpellier_stats))


44.27%
53.84%


## Partie 3 : gestion du temps

> Comme vous l’avez sans-doute remarqué, les informations sur les parkings fournies sont datées.  
> Les données sont valables au moment de la requête. Bien entendu, le nombre de places disponibles évolue 
> au cours du temps en fonction du nombre de voitures qui entrent ou sortent du parking. Si l’on 
> souhaite suivre l’évolution de l’occupation d’un parking au cours du temps (par exemple sur une 
> journée), il convient tout d’abord de définir une période d’échantillonnage Te (intervalle de temps 
> entre deux mesures) puis, de lancer une requête à chaque période d’échantillonnage Te. Nous avons 
> donc besoin d’automatiser cette tâche (à moins que vous ayez du temps à perdre pour lancer 
> manuellement la requête à chaque Te !!!).  
> Il existe plusieurs façons d’exécuter une tâche de manière répétitive à une cadence Te. Pour ce TP nous 
> allons employer un moyen simple. Pour cela, nous allons utiliser la librairie « time »

### Question 7 - Etudier le programme suivant et expliquer ce qu’il affiche :

In [11]:
import time
temps = int(time.time())
print(temps)

1673459188


Que représente le nombre obtenu ? (indication : epoch, UNIX)

Le nombre obtenu correspond au nombre de secondes qui se sont écoulées depuis le 1er janvier 1970.

### Question 8 -  Ecrire un programme qui permet de récupérer l’occupation du parking « FR_MTP_GARE » toutes les 10 secondes pendant 5 minutes et qui sauvegarde ces données dans un fichier.

### Question 9 - Ecrire un programme qui permet le suivi de l’occupation de tous les parkings de Montpellier en permettant à l’utilisateur de choisir la période d’échantillonnage Te et la durée de l’acquisition. Il pourra également indiquer le nom du fichier dans lequel seront enregistrées les données.

### Question 10 - vous possédez à présent la majorité des outils vous permettant d’acquérir et de sauvegarder des données ! il est tant pour vous de fabriquer votre propre librairie de fonctions afin de faciliter la réalisation de votre projet de SAE (et d’autres projets dans le futur !).

Expliquer votre démarche et commentez correctement vos fonctions. 